In [1]:
import spacy
import csv
import pandas as pd
import numpy as np

### Фразы приветствия

In [2]:
hi = ['Здравствуй', 'здравствуй', 'здравствуйте', 'Здравствуйте', 'Добрый день', 'добрый день', 'Доброе утро', 'доброе утро', 'Добрый вечер', 'добрый вечер', 'Приветствую вас', 'приветствую вас', 'привет', 'Привет', 'Салют', 'салют', 'Приветик', 'приветик', 'Хэлло', 'хэлло', 'Хай', 'хай']

### Фразы прощания

In [3]:
bi = ['до встречи', 'до завтра', 'до вечера', 'всего доброго', 'хорошего дня', 'до свидания', 'пока', 'До встречи', 'До завтра', 'До вечера', 'Всего доброго', 'Хорошего дня', 'До свидания', 'Пока']

In [4]:
table = pd.read_csv("data/test_data.csv") ## data
nlp = spacy.load("ru_core_news_lg") ## russian nlp

In [5]:
table.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [6]:
dinum = table.dlg_id.unique() ## unique dialogs

In [7]:
## insight column for every dialog
for n in dinum:
    table['Insight_{0}'.format(n)] = pd.NaT

In [8]:
text = table.text
for mass in text:
    doc = nlp(mass)
    for ent in doc.ents:
        print(ent.text, ent.label_)

ангелина PER
ангелина PER
инфо ORG
денис дэ е эн PER
ангелина PER
астана LOC
Жесупов PER
Алло дмитрий PER
максим PER
анастасия PER
амо ORG
амо ORG
си эр эм PER
айдар PER
ла ла ла PER
анастасия PER
Ага PER
амо ORG
Хэлп PER
вячеслав PER
максим PER


In [37]:
## delete wrong entity
wrong_ent = ['денис дэ е эн', 'амо', 'си эр эм', 'ла ла ла', 'Хэлп', 'Ага']

In [10]:
ruler = nlp.add_pipe("entity_ruler", before="ner")

In [11]:
patterns = [
    {"label": "PER", "pattern": "дмитрий"},
    {"label": "PER", "pattern": "денис"}
]
for greet in hi:
    patterns.append({"label": "GREET", "pattern": greet})
for b in bi:
    patterns.append({"label": "BYE", "pattern": b})    
ruler.add_patterns(patterns)    

In [12]:
from spacy.language import Language

In [13]:
@Language.component("remove_wrong")
def remove_wrong(doc):
    original_ents = list(doc.ents)
    for ent in doc.ents:
        if ent.text in wrong_ent:
            original_ents.remove(ent)
    doc.ents = original_ents
    return doc

In [14]:
nlp.add_pipe("remove_wrong")

<function __main__.remove_wrong(doc)>

In [15]:
num = 0
t = np.zeros((6,1))
for mass in text:
    num += 1
    doc = nlp(mass)
    for ent in doc.ents:
        if table.role[num-1] == 'manager' and ent.label_ == 'GREET':
            table.loc[t[table.dlg_id[num-1]], 'Insight_{0}'.format(table.dlg_id[num-1])] = 'Dialog {0} manager greets.'.format(table.dlg_id[num-1])
            t[table.dlg_id[num-1]] += 1
        if table.role[num-1] == 'manager' and ent.label_ == 'BYE':
            table.loc[t[table.dlg_id[num-1]], 'Insight_{0}'.format(table.dlg_id[num-1])] = 'Dialog {0} manager said Bye.'.format(table.dlg_id[num-1])
            t[table.dlg_id[num-1]] += 1    
        print(ent.text, ent.label_, table.dlg_id[num-1])

здравствуйте GREET 0
Добрый день GREET 0
ангелина PER 0
до свидания BYE 0
здравствуйте GREET 1
здравствуйте GREET 1
ангелина PER 1
пока BYE 1
инфо ORG 1
денис PER 1
До свидания BYE 1
Здравствуйте GREET 2
здравствуйте GREET 2
ангелина PER 2
Пока BYE 2
астана LOC 2
Жесупов PER 2
Добрый день GREET 3
Алло PER 3
дмитрий PER 3
добрый день GREET 3
максим PER 3
дмитрий PER 3
анастасия PER 3
всего доброго BYE 3
до свидания BYE 3
айдар PER 4
до свидания BYE 4
до свидания BYE 4
здравствуйте GREET 5
анастасия PER 5
вячеслав PER 5
максим PER 5
дмитрий PER 5
до свидания BYE 5


In [17]:
table.head()

,dlg_id,line_n,role,text,Insight_0,Insight_1,Insight_2,Insight_3,Insight_4,Insight_5
0,0,0,client,Алло,Dialog 0 manager greets.,Dialog 1 manager greets.,Dialog 2 manager greets.,Dialog 3 manager greets.,Dialog 4 manager said Bye.,Dialog 5 manager said Bye.
1,0,1,manager,Алло здравствуйте,Dialog 0 manager said Bye.,Dialog 1 manager said Bye.,NaT,Dialog 3 manager said Bye.,NaT,NaT
2,0,2,client,Добрый день,NaT,NaT,NaT,NaT,NaT,NaT
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,NaT,NaT,NaT,NaT,NaT,NaT
4,0,4,client,Ага,NaT,NaT,NaT,NaT,NaT,NaT


In [19]:
from spacy.matcher import Matcher
## verify manager introduce and find name
l = 0
matcher = Matcher(nlp.vocab)
pattern1 = [{"ORTH": "меня"},
            {"ENT_TYPE": "PER"},
            {"ORTH": "зовут"}
           ]
pattern2 = [{"ORTH": "Меня"},
            {"ORTH": "зовут"},
            {"ENT_TYPE": "PER"}
           ]

matcher.add("PROPER_N", [pattern1, pattern2], greedy="LONGEST")
for mass in text:
    l += 1
    doc = nlp(mass)
    matches = matcher(doc)
    matches.sort(key = lambda x: x[1])
    
    for match in matches[:10]:
        table.loc[t[table.dlg_id[l-1]], 'Insight_{0}'.format(table.dlg_id[l-1])] = 'Dialog {0} manager Introduced himself.'.format(table.dlg_id[l-1])
        t[table.dlg_id[l-1]] += 1
        print(match, doc[match[1]:match[2]])
        for ent in doc.ents:
            if ent.label_ == 'PER':
                table.loc[t[table.dlg_id[l-1]], 'Insight_{0}'.format(table.dlg_id[l-1])] = 'Dialog {0} manager name is {1}.'.format(table.dlg_id[l-1], ent.text)
                t[table.dlg_id[l-1]] += 1
                

(471132610767985177, 0, 3) Меня зовут ангелина
(471132610767985177, 0, 3) Меня зовут ангелина
(471132610767985177, 0, 3) Меня зовут ангелина
(471132610767985177, 1, 4) меня максим зовут


In [21]:
## find company name
l1 = 0
matcher2 = Matcher(nlp.vocab)
pattern3 = [{"ORTH": "компания"},
            {"POS": "PROPN"}
           ]
pattern4 = [{"ORTH": "компания"},
            {"IS_ALPHA": True, "OP": "+"},
            {"ORTH": "бизнес"}
           ]
matcher2.add("PROPER_N", [pattern3, pattern4], greedy="LONGEST")
for mass in text:
    l1 += 1
    doc = nlp(mass)
    matches = matcher2(doc)
    matches.sort(key = lambda x: x[1])
    
    for match in matches[:10]:
        table.loc[t[table.dlg_id[l1-1]], 'Insight_{0}'.format(table.dlg_id[l1-1])] = 'Dialog {0} company name is {1}.'.format(table.dlg_id[l1-1], doc[match[1]+1])
        t[table.dlg_id[l1-1]] += 1
        print(match, doc[match[1]+1])

(471132610767985177, 3, 6) диджитал
(471132610767985177, 3, 6) диджитал
(471132610767985177, 3, 6) диджитал
(471132610767985177, 4, 6) китобизнес


In [22]:
table

,dlg_id,line_n,role,text,Insight_0,Insight_1,Insight_2,Insight_3,Insight_4,Insight_5
0,0,0,client,Алло,Dialog 0 manager greets.,Dialog 1 manager greets.,Dialog 2 manager greets.,Dialog 3 manager greets.,Dialog 4 manager said Bye.,Dialog 5 manager said Bye.
1,0,1,manager,Алло здравствуйте,Dialog 0 manager said Bye.,Dialog 1 manager said Bye.,Dialog 2 manager Introduced himself.,Dialog 3 manager said Bye.,NaT,NaT
2,0,2,client,Добрый день,Dialog 0 manager Introduced himself.,Dialog 1 manager Introduced himself.,Dialog 2 manager name is ангелина.,Dialog 3 manager Introduced himself.,NaT,NaT
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,Dialog 0 manager name is ангелина.,Dialog 1 manager name is ангелина.,Dialog 2 company name is диджитал.,Dialog 3 manager name is максим.,NaT,NaT
4,0,4,client,Ага,Dialog 0 company name is диджитал.,Dialog 1 company name is диджитал.,NaT,Dialog 3 company name is китобизнес.,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,NaN,NaN,NaN,NaN,NaN,NaN
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,NaN,NaN,NaN,NaN,NaN,NaN
477,5,140,client,Спасибо спасибо,NaN,NaN,NaN,NaN,NaN,NaN
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
table.head()

,dlg_id,line_n,role,text,Insight_0,Insight_1,Insight_2,Insight_3,Insight_4,Insight_5
0,0,0,client,Алло,Dialog 0 manager greets.,Dialog 1 manager greets.,Dialog 2 manager greets.,Dialog 3 manager greets.,Dialog 4 manager said Bye.,Dialog 5 manager said Bye.
1,0,1,manager,Алло здравствуйте,Dialog 0 manager said Bye.,Dialog 1 manager said Bye.,Dialog 2 manager Introduced himself.,Dialog 3 manager said Bye.,NaT,NaT
2,0,2,client,Добрый день,Dialog 0 manager Introduced himself.,Dialog 1 manager Introduced himself.,Dialog 2 manager name is ангелина.,Dialog 3 manager Introduced himself.,NaT,NaT
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,Dialog 0 manager name is ангелина.,Dialog 1 manager name is ангелина.,Dialog 2 company name is диджитал.,Dialog 3 manager name is максим.,NaT,NaT
4,0,4,client,Ага,Dialog 0 company name is диджитал.,Dialog 1 company name is диджитал.,NaT,Dialog 3 company name is китобизнес.,NaT,NaT


In [24]:
your_word = 'ангелина'

In [26]:
##similarity
import numpy as np
ms = nlp.vocab.vectors.most_similar(
    np.asarray([nlp.vocab.vectors[nlp.vocab.strings[your_word]]]), n=10)
word = [nlp.vocab.strings[w] for w in ms[0][0]]
distances = ms[2]
print(word)

['ангелина', 'таисия', 'аркадьевна', 'альбина', 'львовна', 'матвеевна', 'сергеевна', 'викторовна', 'юрьевна', 'степановна']


In [35]:
# find managers who meets the requirements
for d in dinum:
    if table.loc[0, "Insight_{}".format(d)] == 'Dialog {} manager greets.'.format(d):
        if table.loc[1, "Insight_{}".format(d)] == 'Dialog {} manager said Bye.'.format(d):
            table.loc[t[d], 'Insight_{0}'.format(d)] = 'Manager meets the requirements'

In [36]:
table[:6]

,dlg_id,line_n,role,text,Insight_0,Insight_1,Insight_2,Insight_3,Insight_4,Insight_5
0,0,0,client,Алло,Dialog 0 manager greets.,Dialog 1 manager greets.,Dialog 2 manager greets.,Dialog 3 manager greets.,Dialog 4 manager said Bye.,Dialog 5 manager said Bye.
1,0,1,manager,Алло здравствуйте,Dialog 0 manager said Bye.,Dialog 1 manager said Bye.,Dialog 2 manager Introduced himself.,Dialog 3 manager said Bye.,NaT,NaT
2,0,2,client,Добрый день,Dialog 0 manager Introduced himself.,Dialog 1 manager Introduced himself.,Dialog 2 manager name is ангелина.,Dialog 3 manager Introduced himself.,NaT,NaT
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,Dialog 0 manager name is ангелина.,Dialog 1 manager name is ангелина.,Dialog 2 company name is диджитал.,Dialog 3 manager name is максим.,NaT,NaT
4,0,4,client,Ага,Dialog 0 company name is диджитал.,Dialog 1 company name is диджитал.,NaT,Dialog 3 company name is китобизнес.,NaT,NaT
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,Manager meets the requirements,Manager meets the requirements,NaT,Manager meets the requirements,NaT,NaT
